In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install piq pytorch-msssim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

In [13]:
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.relu = nn.LeakyReLU(0.2, inplace=True)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.relu(out)
        out = self.conv2(out)
        out += residual
        return out


class FeatureEncoderModule(nn.Module):
    def __init__(self, in_channels=1, out_channels=64, num_res_blocks=4):
        super(FeatureEncoderModule, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1)
        self.res_blocks = nn.Sequential(*[ResidualBlock(out_channels) for _ in range(num_res_blocks)])
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=2, padding=1)
        self.relu = nn.LeakyReLU(0.2, inplace=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.res_blocks(x)
        x = self.conv2(x)
        return x


class FeatureDecoderModule(nn.Module):
    def __init__(self, in_channels=64, out_channels=1, num_res_blocks=4):
        super(FeatureDecoderModule, self).__init__()
        self.conv1 = nn.ConvTranspose2d(in_channels, in_channels, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.res_blocks = nn.Sequential(*[ResidualBlock(in_channels) for _ in range(num_res_blocks)])
        self.conv2 = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.relu = nn.LeakyReLU(0.2, inplace=True)
        self.output_activation = nn.Sigmoid()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.res_blocks(x)
        x = self.conv2(x)
        x = self.output_activation(x)
        return x

class FeatureFusionEncryptionModule(nn.Module):
    def __init__(self, channels):
        super(FeatureFusionEncryptionModule, self).__init__()
        self.F_block = ResidualBlock(channels)  # Function F
        self.G_block = ResidualBlock(channels)  # Function G

    def forward(self, Ki, alpha_i):
        EKi = Ki + self.F_block(alpha_i)  # Encrypted Key
        EPi = alpha_i + self.G_block(EKi)  # Encrypted Feature
        return EKi, EPi


class FeatureFusionDecryptionModule(nn.Module):
    def __init__(self, channels):
        super(FeatureFusionDecryptionModule, self).__init__()
        self.F_block = ResidualBlock(channels)  # Function F
        self.G_block = ResidualBlock(channels)  # Function G

    def forward(self, EKi, EPi):
        alpha_i = EPi - self.G_block(EKi)  # Recover alpha
        Ki = EKi - self.F_block(alpha_i)  # Recover feature key
        return Ki, alpha_i



In [14]:
class MedicalImageEncryptionModel(nn.Module):
    def __init__(self, channels=64):
        super(MedicalImageEncryptionModel, self).__init__()
        self.encoder = FeatureEncoderModule()
        self.encryption = FeatureFusionEncryptionModule(channels)
        self.decryption = FeatureFusionDecryptionModule(channels)
        self.decoder = FeatureDecoderModule()

    def forward(self, x):
        # Step 1: Encode Image Features
        Ki = self.encoder(x)

        # Step 2: Generate a random fusion parameter alpha_i (same shape as Ki)
        alpha_i = torch.rand_like(Ki, device=Ki.device)

        # Step 3: Encrypt Features
        EKi, EPi = self.encryption(Ki, alpha_i)

        # Step 4: Decrypt Features
        recovered_Ki, recovered_alpha_i = self.decryption(EKi, EPi)

        # Step 5: Decode the Recovered Features
        reconstructed_x = self.decoder(recovered_Ki)

        return reconstructed_x, Ki, EKi, EPi, recovered_Ki

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_files = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
      img_path = self.image_files[idx]
      image = Image.open(img_path).convert("L")
      filename = os.path.basename(img_path)

      if self.transform:
          image = self.transform(image)

      return image, filename

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

In [ ]:
train_dataset = CustomImageDataset(root_dir='/content/drive/MyDrive/temp_pnu_selection', transform=transform)
test_dataset = CustomImageDataset(root_dir='/content/drive/MyDrive/mtg_processed_final_path', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [ ]:
import torch.nn.functional as F
from pytorch_msssim import SSIM

# Initialize SSIM loss function
ssim_loss_fn = SSIM(data_range=1.0, size_average=True, channel=1)

def hybrid_loss(reconstructed, original, alpha=0.84):
    """
    Computes a weighted sum of (1 - SSIM) and MSE loss.
    Args:
        reconstructed: The reconstructed output from the model.
        original: The original input image.
        alpha: Weight for SSIM loss (should be between 0 and 1).

    Returns:
        Hybrid loss combining SSIM and MSE.
    """
    ssim_loss = 1 - ssim_loss_fn(reconstructed, original)  # Convert SSIM to loss
    mse_loss = F.mse_loss(reconstructed, original)  # MSE Loss

    return alpha * ssim_loss + (1 - alpha) * mse_loss  # Weighted sum


In [15]:
import torch
import torch.optim as optim
from tqdm import tqdm

# Initialize model and move to GPU
model = MedicalImageEncryptionModel().to("cuda")

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 250
log_interval = 10  # Compute loss every 10 epochs

def train_model():
    for epoch in range(1, num_epochs + 1):
        model.train()
        train_loss = 0.0

        for images,_ in tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs}"):
            images = images.to("cuda")
            optimizer.zero_grad()

            # Forward pass
            reconstructed, _, _, _, _ = model(images)

            # Compute loss
            loss = hybrid_loss(reconstructed, images)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        if epoch % log_interval == 0:
            print(f"Epoch [{epoch}/{num_epochs}], Loss: {avg_train_loss:.6f}")

# Run training
train_model()


Epoch 10/250: 100%|██████████| 50/50 [00:08<00:00,  5.70it/s]


Epoch [10/250], Loss: 0.043005


Epoch 20/250: 100%|██████████| 50/50 [00:08<00:00,  5.81it/s]


Epoch [20/250], Loss: 0.024792


Epoch 30/250: 100%|██████████| 50/50 [00:08<00:00,  5.75it/s]


...

Epoch [230/250], Loss: 0.000850


Epoch 240/250: 100%|██████████| 50/50 [00:08<00:00,  5.76it/s]


Epoch [240/250], Loss: 0.000854


Epoch 250/250: 100%|██████████| 50/50 [00:08<00:00,  5.83it/s]

Epoch [250/250], Loss: 0.000853


In [20]:
import torch
import numpy as np
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

def evaluate_model():
    model.eval()
    total_ssim = 0.0
    total_psnr = 0.0
    num_samples = 0

    with torch.no_grad():  # Disable gradient tracking during evaluation
        for images,_ in tqdm(test_loader, desc="Evaluating Model"):
            images = images.to("cuda")

            # Forward pass through the model
            reconstructed_x, Ki, EKi, EPi, recovered_Ki = model(images)

            # Convert tensors to NumPy arrays for SSIM and PSNR computation
            original_np = images.squeeze().cpu().numpy()  # (N, H, W)
            recon_np = reconstructed_x.squeeze().cpu().numpy()  # (N, H, W)

            # Ensure 2D format per image (H, W)
            if original_np.ndim == 3:  # If batch of images (N, H, W)
                for i in range(original_np.shape[0]):
                    min_dim = min(original_np.shape[1], original_np.shape[2])
                    win_size = min(7, min_dim) if min_dim >= 7 else min_dim  # Adjust window size

                    batch_ssim = ssim(original_np[i], recon_np[i], data_range=1.0, win_size=win_size)
                    batch_psnr = psnr(original_np[i], recon_np[i], data_range=1.0)

                    total_ssim += batch_ssim
                    total_psnr += batch_psnr
                    num_samples += 1
            else:  # Single image case
                min_dim = min(original_np.shape[0], original_np.shape[1])
                win_size = min(7, min_dim) if min_dim >= 7 else min_dim  # Adjust window size

                batch_ssim = ssim(original_np, recon_np, data_range=1.0, win_size=win_size)
                batch_psnr = psnr(original_np, recon_np, data_range=1.0)

                total_ssim += batch_ssim
                total_psnr += batch_psnr
                num_samples += 1

    # Compute average SSIM and PSNR
    avg_ssim = total_ssim / num_samples
    avg_psnr = total_psnr / num_samples

    print(f"Average SSIM: {avg_ssim:.4f}")
    print(f"Average PSNR: {avg_psnr:.4f} dB")
    print(num_samples)

# Run Evaluation
evaluate_model()


Evaluating Model: 100%|██████████| 18/18 [00:02<00:00,  8.07it/s]

Average SSIM: 0.9950
Average PSNR: 36.6538 dB
138
